# Script to evaluate inference time

In [4]:
import numpy as np
import h5py

In [5]:
labels = ['j_g', 'j_q', 'j_w', 'j_z', 'j_t']

In [7]:
# input dataset
import glob
X_hlf = np.array([])
Y = np.array([])
file_50 = h5py.File("../data/jetImage_50_VAL.h5","r")
file_100 = h5py.File("../data/jetImage_7_100p_50000_60000.h5","r")
file_150 = h5py.File("../data/jetImage_7_150p_10000_20000.h5","r")

HLF_50 = np.array(file_50.get('jets')[:,[12, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 52]])
Y_50 = np.array(file_50.get('jets')[0:,-6:-1])
Image_50 = np.array(file_50.get('jetImage'))
Image_50 = Image_50.reshape((Image_50.shape[0], Image_50.shape[1], Image_50.shape[2], 1))
List_50 =  np.array(file_50.get('jetConstituentList'))

Y_100 = np.array(file_100.get('jets')[0:,-6:-1])
List_100 =  np.array(file_50.get('jetConstituentList'))

Y_150 = np.array(file_100.get('jets')[0:,-6:-1])
List_150 =  np.array(file_50.get('jetConstituentList'))


OSError: Unable to open file (unable to open file: name = 'data/jetImage_50_VAL.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

# DNN

In [2]:
# keras imports
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Conv2D, Dropout, Flatten
from keras.layers import Concatenate, BatchNormalization, Activation
from keras.layers import MaxPooling2D, MaxPooling3D, GRU
from keras.utils import plot_model
from keras import regularizers
from keras import backend as K
from keras import metrics
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
from keras.regularizers import l1

Using TensorFlow backend.


In [3]:
#optmizer = 'adam'
# best model from optimization
DNN_neurons = 80
DNN_layers = 3
DNN_activation = 'elu'
dropout = 0.10
batch_size = 50
n_epochs = 500

In [4]:
#  model
def myModel():
    inputArray = Input(shape=(input_shape,))
    x = Dense(DNN_neurons, activation=DNN_activation, 
              kernel_initializer='lecun_uniform', name='dense_0')(inputArray)
    x = Dropout(dropout)(x)
    ####
    for i in range(1,DNN_layers):
        x = Dense(DNN_neurons, activation=DNN_activation, 
                  kernel_initializer='lecun_uniform', name='dense_%i' %i)(x)
        x = Dropout(dropout)(x)

    output = Dense(5, activation='softmax', kernel_initializer='lecun_uniform', 
                   name = 'output_softmax')(x)
    ####
    model = Model(inputs=inputArray, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    return model

In [5]:
input_shape = len([12, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 52])
thisModel = myModel()
thisModel.summary()

W0719 12:27:05.257584 4549944768 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0719 12:27:05.272451 4549944768 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0719 12:27:05.276516 4549944768 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0719 12:27:05.310244 4549944768 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0719 12:27:05.321657 4549944768 deprecati

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 16)                0         
_________________________________________________________________
dense_0 (Dense)              (None, 80)                1360      
_________________________________________________________________
dropout_1 (Dropout)          (None, 80)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 80)                6480      
_________________________________________________________________
dropout_2 (Dropout)          (None, 80)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 80)                6480      
_________________________________________________________________
dropout_3 (Dropout)          (None, 80)                0         
__________

# CNN

In [6]:
CNN_filters = 10
CNN_filter_size = 3
CNN_MaxPool_size = 5
CNN_layers = 1
CNN_activation = 'elu'
DNN_neurons = 50
DNN_layers = 3
DNN_activation = 'elu'
dropout = 0.1
batch_size = 500
n_epochs = 500
labels = ['j_g', 'j_q', 'j_w', 'j_z', 'j_t']
nParticles = 50

In [7]:
#  model
def myModel():
    inputImage = Input(shape=(image_shape))
    x = Conv2D(CNN_filters, kernel_size=(CNN_filter_size,CNN_filter_size), 
               data_format="channels_last", strides=(1, 1), padding="same", input_shape=image_shape,
               kernel_initializer='lecun_uniform', name='cnn2D_0')(inputImage)
    x = BatchNormalization()(x)
    x = Activation(CNN_activation)(x)
    x = MaxPooling2D( pool_size = (CNN_MaxPool_size,CNN_MaxPool_size))(x)
    x = Dropout(dropout)(x)
    for i in range(1,CNN_layers):
        x = Conv2D(CNN_filters, kernel_size=(CNN_filter_size,CNN_filter_size), 
                   data_format="channels_last", strides=(1, 1), padding="same", input_shape=image_shape,
                   kernel_initializer='lecun_uniform', name='cnn2D_%i' %i)(x)
        x = BatchNormalization()(x)
        x = Activation(CNN_activation)(x)
        #x = MaxPooling2D( pool_size = (CNN_MaxPool_size,CNN_MaxPool_size))(x)
        x = Dropout(dropout)(x)
        
    ####
    x = Flatten()(x)
    #
    for i in range(DNN_layers):
        x = Dense(DNN_neurons, activation=DNN_activation, 
                  kernel_initializer='lecun_uniform', name='dense_%i' %i)(x)
        x = Dropout(dropout)(x)
    #
    output = Dense(5, activation='softmax', kernel_initializer='lecun_uniform', 
                   name = 'output_softmax')(x)
    ####
    model = Model(inputs=inputImage, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

    return model

In [8]:
img_rows = 100#X.shape[1]
img_cols = 100#X.shape[2]
image_shape = (img_rows, img_cols, 1)

In [9]:
thisModel = myModel()
thisModel.summary()

W0719 12:27:07.751152 4549944768 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0719 12:27:07.816950 4549944768 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100, 100, 1)       0         
_________________________________________________________________
cnn2D_0 (Conv2D)             (None, 100, 100, 10)      100       
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 100, 10)      40        
_________________________________________________________________
activation_1 (Activation)    (None, 100, 100, 10)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 10)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 20, 20, 10)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4000)              0         
__________

# GRU

In [10]:
nParticles = 50
GRU_units= 50
DNN_neurons = 40
DNN_layers = 3
DNN_activation = 'relu'
dropout = 0.22
batch_size = 500
n_epochs = 50
#n_epochs = 1
labels = ['j_g', 'j_q', 'j_w', 'j_z', 'j_t']
input_shape = (nParticles,16)

In [11]:
#  model
def myModel():
    inputArray = Input(shape=(input_shape))
    x = GRU(GRU_units, activation='tanh',
            recurrent_activation='hard_sigmoid', name='gru')(inputArray)
    x = Dropout(dropout)(x)
    ####
    for i in range(0,DNN_layers):
        x = Dense(DNN_neurons, activation=DNN_activation, 
                  kernel_initializer='lecun_uniform', name='dense_%i' %i)(x)
        x = Dropout(dropout)(x)
    #
    output = Dense(5, activation='softmax', kernel_initializer='lecun_uniform', 
                   name = 'output_softmax')(x)
    ####
    model = Model(inputs=inputArray, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    return model

In [12]:
thisModel = myModel()
thisModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 50, 16)            0         
_________________________________________________________________
gru (GRU)                    (None, 50)                10050     
_________________________________________________________________
dropout_8 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_0 (Dense)              (None, 40)                2040      
_________________________________________________________________
dropout_9 (Dropout)          (None, 40)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 40)                1640      
_________________________________________________________________
dropout_10 (Dropout)         (None, 40)                0         
__________

# JEDI-net

In [1]:
import torch
import torch.nn as nn
from torch.autograd.variable import *
import torch.optim as optim
import itertools

In [2]:
class GraphNet(nn.Module):
    def __init__(self, n_constituents, n_targets, params, hidden, De, Do, 
                 fr_activation=0, fo_activation=0, fc_activation=0, optimizer = 0, verbose = False):
        super(GraphNet, self).__init__()
        self.hidden = hidden
        self.P = len(params)
        self.N = n_constituents
        self.Nr = self.N * (self.N - 1)
        self.Dr = 0
        self.De = De
        self.Dx = 0
        self.Do = Do
        self.n_targets = n_targets
        self.fr_activation = fr_activation
        self.fo_activation = fo_activation
        self.fc_activation = fc_activation
        self.optimizer = optimizer
        self.verbose = verbose
        self.assign_matrices()

        self.Ra = torch.ones(self.Dr, self.Nr)
        self.fr1 = nn.Linear(2 * self.P + self.Dr, hidden)
        self.fr2 = nn.Linear(hidden, int(hidden/2))
        self.fr3 = nn.Linear(int(hidden/2), self.De)
        self.fo1 = nn.Linear(self.P + self.Dx + self.De, hidden)
        self.fo2 = nn.Linear(hidden, int(hidden/2))
        self.fo3 = nn.Linear(int(hidden/2), self.Do)
        self.fc1 = nn.Linear(self.Do * self.N, hidden)
        self.fc2 = nn.Linear(hidden, int(hidden/2))
        self.fc3 = nn.Linear(int(hidden/2), self.n_targets)

    def assign_matrices(self):
        self.Rr = torch.zeros(self.N, self.Nr)
        self.Rs = torch.zeros(self.N, self.Nr)
        receiver_sender_list = [i for i in itertools.product(range(self.N), range(self.N)) if i[0]!=i[1]]
        for i, (r, s) in enumerate(receiver_sender_list):
            self.Rr[r, i] = 1
            self.Rs[s, i] = 1
        self.Rr = Variable(self.Rr)
        self.Rs = Variable(self.Rs)

    def forward(self, x):
        Orr = self.tmul(x, self.Rr)
        Ors = self.tmul(x, self.Rs)
        B = torch.cat([Orr, Ors], 1)
        ### First MLP ###
        B = torch.transpose(B, 1, 2).contiguous()
        if self.fr_activation ==2:
            B = nn.functional.selu(self.fr1(B.view(-1, 2 * self.P + self.Dr)))
            B = nn.functional.selu(self.fr2(B))
            E = nn.functional.selu(self.fr3(B).view(-1, self.Nr, self.De))            
        elif self.fr_activation ==1:
            B = nn.functional.elu(self.fr1(B.view(-1, 2 * self.P + self.Dr)))
            B = nn.functional.elu(self.fr2(B))
            E = nn.functional.elu(self.fr3(B).view(-1, self.Nr, self.De))
        else:
            B = nn.functional.relu(self.fr1(B.view(-1, 2 * self.P + self.Dr)))
            B = nn.functional.relu(self.fr2(B))
            E = nn.functional.relu(self.fr3(B).view(-1, self.Nr, self.De))
        del B
        E = torch.transpose(E, 1, 2).contiguous()
        Ebar = self.tmul(E, torch.transpose(self.Rr, 0, 1).contiguous())
        del E
        C = torch.cat([x, Ebar], 1)
        del Ebar
        C = torch.transpose(C, 1, 2).contiguous()
        ### Second MLP ###
        if self.fo_activation ==2:
            C = nn.functional.selu(self.fo1(C.view(-1, self.P + self.Dx + self.De)))
            C = nn.functional.selu(self.fo2(C))
            O = nn.functional.selu(self.fo3(C).view(-1, self.N, self.Do))
        elif self.fo_activation ==1:
            C = nn.functional.elu(self.fo1(C.view(-1, self.P + self.Dx + self.De)))
            C = nn.functional.elu(self.fo2(C))
            O = nn.functional.elu(self.fo3(C).view(-1, self.N, self.Do))
        else:
            C = nn.functional.relu(self.fo1(C.view(-1, self.P + self.Dx + self.De)))
            C = nn.functional.relu(self.fo2(C))
            O = nn.functional.relu(self.fo3(C).view(-1, self.N, self.Do))
        del C
        ### Classification MLP ###
        if self.fc_activation ==2:
            N = nn.functional.selu(self.fc1(O.view(-1, self.Do * self.N)))
            N = nn.functional.selu(self.fc2(N))       
        elif self.fc_activation ==1:
            N = nn.functional.elu(self.fc1(O.view(-1, self.Do * self.N)))
            N = nn.functional.elu(self.fc2(N))
        else:
            N = nn.functional.relu(self.fc1(O.view(-1, self.Do * self.N)))
            N = nn.functional.relu(self.fc2(N))
        del O
        #N = nn.functional.relu(self.fc3(N))
        N = self.fc3(N)
        return N

    def tmul(self, x, y):  #Takes (I * J * K)(K * L) -> I * J * L 
        x_shape = x.size()
        y_shape = y.size()
        return torch.mm(x.view(-1, x_shape[2]), y).view(-1, x_shape[1], y_shape[1])

In [3]:
# ### Prepare Dataset                                                                                                                                                  
nParticles = 100
x = []
x.append(30) # hinned nodes                                                                                                                                            
x.append(10) # De                                                                                                                                                      
x.append(10) # Do                                                                                                                                                      
x.append(1) # fr_activation_index                                                                                                                                      
x.append(1) # fo_activation_index                                                                                                                                      
x.append(1) # fc_activation_index                                                                                                                                      
x.append(0) # optmizer_index           

In [4]:
params = ['j1_px', 'j1_py' , 'j1_pz' , 'j1_e' , 'j1_erel' , 'j1_pt' , 'j1_ptrel', 'j1_eta' , 'j1_etarel' , 
          'j1_etarot' , 'j1_phi' , 'j1_phirel' , 'j1_phirot', 'j1_deltaR' , 'j1_costheta' , 'j1_costhetarel']

In [7]:
mymodel = GraphNet(nParticles, len(labels), params, int(x[0]), int(x[1]), int(x[2]), 
                       int(x[3]),  int(x[4]),  int(x[5]), int(x[6]), 0)

In [8]:
print(mymodel)
trainablePars = sum(p.numel() for p in mymodel.parameters() if p.requires_grad)
print('\nTrainable parameters:', trainablePars)

GraphNet(
  (fr1): Linear(in_features=32, out_features=30, bias=True)
  (fr2): Linear(in_features=30, out_features=15, bias=True)
  (fr3): Linear(in_features=15, out_features=10, bias=True)
  (fo1): Linear(in_features=26, out_features=30, bias=True)
  (fo2): Linear(in_features=30, out_features=15, bias=True)
  (fo3): Linear(in_features=15, out_features=10, bias=True)
  (fc1): Linear(in_features=1000, out_features=30, bias=True)
  (fc2): Linear(in_features=30, out_features=15, bias=True)
  (fc3): Linear(in_features=15, out_features=5, bias=True)
)

Trainable parameters: 33625


# JEDI-net with Sum over O

In [9]:
class GraphNet(nn.Module):
    def __init__(self, n_constituents, n_targets, params, hidden, De, Do, 
                 fr_activation=0, fo_activation=0, fc_activation=0, optimizer = 0, verbose = False):
        super(GraphNet, self).__init__()
        self.hidden = hidden
        self.P = len(params)
        self.N = n_constituents
        self.Nr = self.N * (self.N - 1)
        self.Dr = 0
        self.De = De
        self.Dx = 0
        self.Do = Do
        self.n_targets = n_targets
        self.fr_activation = fr_activation
        self.fo_activation = fo_activation
        self.fc_activation = fc_activation
        self.optimizer = optimizer
        self.verbose = verbose
        self.assign_matrices()

        self.Ra = torch.ones(self.Dr, self.Nr)
        self.fr1 = nn.Linear(2 * self.P + self.Dr, hidden)
        self.fr2 = nn.Linear(hidden, int(hidden/2))
        self.fr3 = nn.Linear(int(hidden/2), self.De)
        self.fo1 = nn.Linear(self.P + self.Dx + self.De, hidden)
        self.fo2 = nn.Linear(hidden, int(hidden/2))
        self.fo3 = nn.Linear(int(hidden/2), self.Do)
        self.fc1 = nn.Linear(self.Do, hidden)
        self.fc2 = nn.Linear(hidden, int(hidden/2))
        self.fc3 = nn.Linear(int(hidden/2), self.n_targets)

    def assign_matrices(self):
        self.Rr = torch.zeros(self.N, self.Nr)
        self.Rs = torch.zeros(self.N, self.Nr)
        receiver_sender_list = [i for i in itertools.product(range(self.N), range(self.N)) if i[0]!=i[1]]
        for i, (r, s) in enumerate(receiver_sender_list):
            self.Rr[r, i] = 1
            self.Rs[s, i] = 1
        self.Rr = Variable(self.Rr)
        self.Rs = Variable(self.Rs)

    def forward(self, x):
        Orr = self.tmul(x, self.Rr)
        Ors = self.tmul(x, self.Rs)
        B = torch.cat([Orr, Ors], 1)
        ### First MLP ###
        B = torch.transpose(B, 1, 2).contiguous()
        if self.fr_activation ==2:
            B = nn.functional.selu(self.fr1(B.view(-1, 2 * self.P + self.Dr)))
            B = nn.functional.selu(self.fr2(B))
            E = nn.functional.selu(self.fr3(B).view(-1, self.Nr, self.De))            
        elif self.fr_activation ==1:
            B = nn.functional.elu(self.fr1(B.view(-1, 2 * self.P + self.Dr)))
            B = nn.functional.elu(self.fr2(B))
            E = nn.functional.elu(self.fr3(B).view(-1, self.Nr, self.De))
        else:
            B = nn.functional.relu(self.fr1(B.view(-1, 2 * self.P + self.Dr)))
            B = nn.functional.relu(self.fr2(B))
            E = nn.functional.relu(self.fr3(B).view(-1, self.Nr, self.De))
        del B
        E = torch.transpose(E, 1, 2).contiguous()
        Ebar = self.tmul(E, torch.transpose(self.Rr, 0, 1).contiguous())
        del E
        C = torch.cat([x, Ebar], 1)
        del Ebar
        C = torch.transpose(C, 1, 2).contiguous()
        ### Second MLP ###
        if self.fo_activation ==2:
            C = nn.functional.selu(self.fo1(C.view(-1, self.P + self.Dx + self.De)))
            C = nn.functional.selu(self.fo2(C))
            O = nn.functional.selu(self.fo3(C).view(-1, self.N, self.Do))
        elif self.fo_activation ==1:
            C = nn.functional.elu(self.fo1(C.view(-1, self.P + self.Dx + self.De)))
            C = nn.functional.elu(self.fo2(C))
            O = nn.functional.elu(self.fo3(C).view(-1, self.N, self.Do))
        else:
            C = nn.functional.relu(self.fo1(C.view(-1, self.P + self.Dx + self.De)))
            C = nn.functional.relu(self.fo2(C))
            O = nn.functional.relu(self.fo3(C).view(-1, self.N, self.Do))
        del C
        ## sum over the O matrix  
        O = torch.sum( O, dim=1)
        ### Classification MLP ###
        if self.fc_activation ==2:
            N = nn.functional.selu(self.fc1(O.view(-1, self.Do)))
            N = nn.functional.selu(self.fc2(N))       
        elif self.fc_activation ==1:
            N = nn.functional.elu(self.fc1(O.view(-1, self.Do)))
            N = nn.functional.elu(self.fc2(N))
        else:
            N = nn.functional.relu(self.fc1(O.view(-1, self.Do)))
            N = nn.functional.relu(self.fc2(N))
        del O
        #N = nn.functional.relu(self.fc3(N))
        N = self.fc3(N)
        return N

    def tmul(self, x, y):  #Takes (I * J * K)(K * L) -> I * J * L 
        x_shape = x.size()
        y_shape = y.size()
        return torch.mm(x.view(-1, x_shape[2]), y).view(-1, x_shape[1], y_shape[1])

In [10]:
# ### Prepare Dataset                                                                                                                                                  
nParticles = 150
x = []
x.append(50) # hinned nodes                                                                                                                                            
x.append(14) # De                                                                                                                                                      
x.append(12) # Do                                                                                                                                                      
x.append(2) # fr_activation_index                                                                                                                                      
x.append(2) # fo_activation_index                                                                                                                                      
x.append(2) # fc_activation_index                                                                                                                                      
x.append(0) # optmizer_index           

In [11]:
mymodel = GraphNet(nParticles, len(labels), params, int(x[0]), int(x[1]), int(x[2]), 
                       int(x[3]),  int(x[4]),  int(x[5]), int(x[6]), 0)

In [12]:
print(mymodel)
trainablePars = sum(p.numel() for p in mymodel.parameters() if p.requires_grad)
print('\nTrainable parameters:', trainablePars)

GraphNet(
  (fr1): Linear(in_features=32, out_features=50, bias=True)
  (fr2): Linear(in_features=50, out_features=25, bias=True)
  (fr3): Linear(in_features=25, out_features=14, bias=True)
  (fo1): Linear(in_features=30, out_features=50, bias=True)
  (fo2): Linear(in_features=50, out_features=25, bias=True)
  (fo3): Linear(in_features=25, out_features=12, bias=True)
  (fc1): Linear(in_features=12, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=25, bias=True)
  (fc3): Linear(in_features=25, out_features=5, bias=True)
)

Trainable parameters: 8481
